In [3]:
%load_ext autoreload
%autoreload 2

import json
import os, sys
import requests
from solana.rpc.api import Client, Signature
from solders.pubkey import Pubkey
import datetime as dt
import pandas as pd
import numpy as np
from solana.rpc.types import TokenAccountOpts
import polars as pl

In [4]:
from atlas_sol import AtlasSol
from tqdm import tqdm
import base58
from solana.rpc.api import Client
from spl.token.constants import TOKEN_PROGRAM_ID

CHAINSTAKE_SOLONA_HTTPS = "https://solana-mainnet.core.chainstack.com/2fa69914c087050cc7b9887511cd7d35"
CHAINSTAKE_SOLONA_WS = "wss://solana-mainnet.core.chainstack.com/2fa69914c087050cc7b9887511cd7d35"
client = Client(CHAINSTAKE_SOLONA_WS)
asol = AtlasSol(CHAINSTAKE_SOLONA_HTTPS)

In [5]:
engine = AtlasSol.GetPostgresEngine()

In [ ]:
import asyncio
from asyncstdlib import enumerate
from solana.rpc.websocket_api import connect

dfs = []
MSG_COUNT = 5_000
i = 0
async with connect(CHAINSTAKE_SOLONA_WS) as websocket:
    websocket.max_size = 2 ** 40
    await websocket.block_subscribe(
        commitment="finalized",
        encoding="binary",
        max_supported_transaction_version=0
    )
    first_resp = await websocket.recv()  # Wait for a message
    subscription_id = first_resp[0].result
    while True:
        try:
            subscription_id = first_resp[0].result
            message = await websocket.recv()  # Wait for a message
            now = dt.datetime.now()
            df_t = asol.getTxFrame(
                block=message[0].result.value.block,
                raise_on_error=False
            )
            df_t.write_database(
                table_name="tx_detail_v2",
                connection=engine,
                if_table_exists="append",
            )
            print(f"Msg recv: {now}, len: {len(df_t)}")
            i+=1
            if i == MSG_COUNT:
                break
        except Exception as e:
            print(f"Error receiving message: {e}")
    await websocket.logs_unsubscribe(subscription_id)

Msg recv: 2024-12-08 19:08:52.561827, len: 124
Msg recv: 2024-12-08 19:08:53.099037, len: 101
Msg recv: 2024-12-08 19:08:54.043861, len: 127
Msg recv: 2024-12-08 19:08:54.711046, len: 99
Msg recv: 2024-12-08 19:08:55.147432, len: 97
Msg recv: 2024-12-08 19:08:55.682354, len: 64
Msg recv: 2024-12-08 19:08:56.381574, len: 128
Msg recv: 2024-12-08 19:08:56.970778, len: 158
Msg recv: 2024-12-08 19:08:57.625401, len: 138
Msg recv: 2024-12-08 19:08:58.256651, len: 122
Msg recv: 2024-12-08 19:08:59.114079, len: 179
Msg recv: 2024-12-08 19:08:59.931568, len: 134
Msg recv: 2024-12-08 19:09:00.508166, len: 108
Msg recv: 2024-12-08 19:09:01.032491, len: 118
Msg recv: 2024-12-08 19:09:01.628218, len: 140
Msg recv: 2024-12-08 19:09:02.261346, len: 138
Msg recv: 2024-12-08 19:09:02.771171, len: 104
Msg recv: 2024-12-08 19:09:03.750189, len: 95
Msg recv: 2024-12-08 19:09:04.488391, len: 148
Msg recv: 2024-12-08 19:09:05.065605, len: 132
Msg recv: 2024-12-08 19:09:05.654753, len: 110
Msg recv: 2024-12

In [27]:
q = """
SELECT * FROM public.tx_detail_v2
"""
df = pl.read_database(
    q,
    connection=engine
)
df


signature,owner,account,mint,wallet_prebalance,wallet_postbalance,owner_cp,account_cp,wallet_prebalance_cp,wallet_postbalance_cp,mint_cp,wallet_prebalance_sold,wallet_postbalance_sold,account_sold,timestamp,parent_slot
str,str,str,str,f64,f64,str,str,f64,f64,str,f64,f64,str,datetime[μs],i64
"""3XSebZeC1tpEJLaWRrB2BwhQydeGnY…","""5Q544fKrFoe6tsEbD7S8EmxGTJYAKt…","""CEsLEVSErrnncyY2AhEagS2kAC8Xhc…","""6fqSjchG9eL796uCk8Bdh6GA8aYN76…",75258.83305,75258.978163,"""4aDb9teSkXFXj67T66XMbsv4mMH1Fz…","""5MGEroCREpe2w7HUe9hbK4tSTVwMn7…",7.178449,7.033336,"""So1111111111111111111111111111…",0.113093,0.113093,"""6xGwJqWgMhws9ATjXDhYio3QM5HrET…",2024-12-08 23:45:17,306280192
"""2B1TqsCQcsm9yQCPz4z1QhpXQQQA6h…","""5Q544fKrFoe6tsEbD7S8EmxGTJYAKt…","""APU2BpNbsjsxFqxwj86wuN6rP4wAFp…","""xv8yd91ckrxH52z67HLaerZXFRubF4…",7.4503e7,7.4503e7,"""5CCJb1oYP19Dziksn6s9BhZvWanf21…","""BeSh6wn5aUg9zzFHVJfJNE2dkhEVWj…",6236.027663,6249.660853,"""So1111111111111111111111111111…",54.511868,54.511878,"""GSoT84vkzAdU1hkBWeEZcL19rMeCLJ…",2024-12-08 23:45:17,306280192
"""2XcNqF5c2Tzj2cHtwG7uYr8EvoxxF7…","""5Q544fKrFoe6tsEbD7S8EmxGTJYAKt…","""APU2BpNbsjsxFqxwj86wuN6rP4wAFp…","""xv8yd91ckrxH52z67HLaerZXFRubF4…",7.4503e7,7.4503e7,"""5CCJb1oYP19Dziksn6s9BhZvWanf21…","""BeSh6wn5aUg9zzFHVJfJNE2dkhEVWj…",6249.660853,6263.294037,"""So1111111111111111111111111111…",54.511878,54.511888,"""GSoT84vkzAdU1hkBWeEZcL19rMeCLJ…",2024-12-08 23:45:17,306280192
"""4PMpbfLMyoPZHUZCyEDhXhwHXyJ5GF…","""5Q544fKrFoe6tsEbD7S8EmxGTJYAKt…","""APU2BpNbsjsxFqxwj86wuN6rP4wAFp…","""xv8yd91ckrxH52z67HLaerZXFRubF4…",7.4503e7,7.4503e7,"""5CCJb1oYP19Dziksn6s9BhZvWanf21…","""BeSh6wn5aUg9zzFHVJfJNE2dkhEVWj…",6263.294037,6276.927217,"""So1111111111111111111111111111…",54.511888,54.511898,"""GSoT84vkzAdU1hkBWeEZcL19rMeCLJ…",2024-12-08 23:45:17,306280192
"""4t2S7Qk7zbV7qaZjregMQ3n66Dbv89…","""5Q544fKrFoe6tsEbD7S8EmxGTJYAKt…","""APU2BpNbsjsxFqxwj86wuN6rP4wAFp…","""xv8yd91ckrxH52z67HLaerZXFRubF4…",7.4503e7,7.4503e7,"""5CCJb1oYP19Dziksn6s9BhZvWanf21…","""BeSh6wn5aUg9zzFHVJfJNE2dkhEVWj…",6276.927217,6290.560392,"""So1111111111111111111111111111…",54.511898,54.511908,"""GSoT84vkzAdU1hkBWeEZcL19rMeCLJ…",2024-12-08 23:45:17,306280192
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""mb1Yz9JpgopnCw2QPtg97CoUNJRKsh…","""7vi5dyBk6s7nj3HPmSN35Ts2ZXkaKK…","""27V4G3DHF4x5mybjED1WNTN6Cy6WSV…","""orcaEKTdK7LKz57vaAYr9QeNsVEPfi…",14144.126637,14966.585376,"""9RJLGgKga286QDQtBAA2bfYSpQ6bNP…","""Dhbx3k97ffBEyeXisynTejX84dkNzr…",43081.288408,42258.829669,"""EPjFWdd5AufqSSqeM2qN1xzybapC8G…",1.2105e6,1.2054e6,"""HMw6T5r7eaXq6WgDCpZYJn5bBGziNH…",2024-12-08 23:56:30,306281831
"""3LKJ5Tqme88akZB14bndktaXqd2dPS…","""7nuFxQAu8SSD81PaXUfiTCLRRBmCoe…","""AjaA3x3i4VgtaQonmX6wAucANhr79Z…","""4Bo121MZ4C16XEsz47MbmuyEx1CFXe…",49628.798607,0.0,"""5Q544fKrFoe6tsEbD7S8EmxGTJYAKt…","""FfirHqUoT1q7tjfjGTAJdnicQGmYW1…",6.4145e7,6.4195e7,"""4Bo121MZ4C16XEsz47MbmuyEx1CFXe…",6.4145e7,6.4195e7,"""FfirHqUoT1q7tjfjGTAJdnicQGmYW1…",2024-12-08 23:56:30,306281831
"""3ZDXYYxZALibm2ZBShRyS6rqQ17AAq…","""DCBiiQf9SzZeUB8mo9LrTf13BzGja6…","""4q2MvCf7cQbT4B1YPRsZAmB6fVJiGZ…","""GfHjaxzmPQH1FriETck6Hivr9j9wDQ…",2.1492e7,2.1492e7,"""GpMZbSM2GgvTKHJirzeGfMFoaZ8UR2…","""7bBf1PhrM3yk8MiszdCNfMeFm1zGtf…",2.2347e7,2.2347e7,"""GfHjaxzmPQH1FriETck6Hivr9j9wDQ…",2.2347e7,2.2347e7,"""7bBf1PhrM3yk8MiszdCNfMeFm1zGtf…",2024-12-08 23:56:30,306281831


In [31]:
df_price = await asol.getPrices()
df_token = AtlasSol.getTokenList()
df_price = df_price.with_columns(
    pl.col("symbol").str.split("/").list.first().alias("symbol"),
).join(
    df_token.select("address", "symbol", 'name'),
    on="symbol",
    how="inner"
)
df_price.head(4)

key,product_account_key,symbol,status,price,timestamp,ci,address,name
str,str,str,str,f64,i64,f64,str,str
"""H6ARHf6YXhGYeQfUzQNGk6rDNnLBQK…","""ALP8SdU9oARYVLgLR7LrqMNCYBnhtn…","""SOL""","""TRADING""",236.60505,1733702722,0.14667,"""So1111111111111111111111111111…","""Wrapped SOL"""
"""GVXRSBjFk6e6J3NbVPXohDJetcTjae…","""4aDoSXJ5o3AuvL7QFeR6h44jALQfTm…","""BTC""","""TRADING""",100934.756868,1733702722,52.823864,"""9n4nbM75f5Ui33ZbPYXn59EwSgE8CG…","""Wrapped Bitcoin (Sollet)"""
"""ETp9eKXVv1dWwHSpsXRUuXHmw24PwR…","""HyEB4Goiv7QyfFStaBn49JqQzSTV1y…","""FIDA""","""TRADING""",0.3905978,1733702722,0.000628,"""EchesyfXePKdLtoiZSL8pBe8Myagyy…","""Bonfida"""
"""AnLf8tVYCM816gmBjiy8n53eXKKEDy…","""EjtaES3pbMCaNrVFtZ5sT36WPXAnAR…","""RAY""","""TRADING""",4.955418,1733702722,0.005477,"""4k3Dyjzvzp8eMZWUXbBCjEvwSkkk59…","""Raydium"""


In [39]:
df_res = df.join(
    df_price.select(
        [
            pl.col("address").alias("mint"),
            pl.col("price").alias("mint_price"),
            pl.col("name").alias("mint_name"),
        ]
    ),
    on="mint",
)
df_res = df_res.join(
    df_price.select(
        [
            pl.col("address").alias("mint_cp"),
            pl.col("price").alias("mint_cp_price"),
            pl.col("name").alias("mint_cp_name"),
        ]
    ),
    on="mint_cp",
)
df_res

signature,owner,account,mint,wallet_prebalance,wallet_postbalance,owner_cp,account_cp,wallet_prebalance_cp,wallet_postbalance_cp,mint_cp,wallet_prebalance_sold,wallet_postbalance_sold,account_sold,timestamp,parent_slot,mint_price,mint_name,mint_cp_price,mint_cp_name
str,str,str,str,f64,f64,str,str,f64,f64,str,f64,f64,str,datetime[μs],i64,f64,str,f64,str
"""2887tRV2FjyfcWc7vCQ3raJyrcosPy…","""3khmkbjgxisdw2uT9H8pdgxSGsYwYW…","""99rs3v5754GMQkf2KBf1sL2tj6q95f…","""So1111111111111111111111111111…",1.54,1.32,"""5Q544fKrFoe6tsEbD7S8EmxGTJYAKt…","""67kvwjMKT2nmPvWMQmLraTWK7y8Jch…",173.635275,173.855275,"""So1111111111111111111111111111…",173.635275,173.855275,"""67kvwjMKT2nmPvWMQmLraTWK7y8Jch…",2024-12-08 23:45:17,306280192,236.60505,"""Wrapped SOL""",236.60505,"""Wrapped SOL"""
"""2887tRV2FjyfcWc7vCQ3raJyrcosPy…","""3khmkbjgxisdw2uT9H8pdgxSGsYwYW…","""99rs3v5754GMQkf2KBf1sL2tj6q95f…","""So1111111111111111111111111111…",1.54,1.32,"""5Q544fKrFoe6tsEbD7S8EmxGTJYAKt…","""67kvwjMKT2nmPvWMQmLraTWK7y8Jch…",173.635275,173.855275,"""So1111111111111111111111111111…",173.635275,173.855275,"""67kvwjMKT2nmPvWMQmLraTWK7y8Jch…",2024-12-08 23:45:17,306280192,236.60505,"""Wrapped SOL""",236.60505,"""Wrapped SOL"""
"""2887tRV2FjyfcWc7vCQ3raJyrcosPy…","""3khmkbjgxisdw2uT9H8pdgxSGsYwYW…","""99rs3v5754GMQkf2KBf1sL2tj6q95f…","""So1111111111111111111111111111…",1.54,1.32,"""5Q544fKrFoe6tsEbD7S8EmxGTJYAKt…","""67kvwjMKT2nmPvWMQmLraTWK7y8Jch…",173.635275,173.855275,"""So1111111111111111111111111111…",173.635275,173.855275,"""67kvwjMKT2nmPvWMQmLraTWK7y8Jch…",2024-12-08 23:45:17,306280192,236.60505,"""Wrapped SOL""",236.60505,"""Wrapped SOL"""
"""2887tRV2FjyfcWc7vCQ3raJyrcosPy…","""3khmkbjgxisdw2uT9H8pdgxSGsYwYW…","""99rs3v5754GMQkf2KBf1sL2tj6q95f…","""So1111111111111111111111111111…",1.54,1.32,"""5Q544fKrFoe6tsEbD7S8EmxGTJYAKt…","""67kvwjMKT2nmPvWMQmLraTWK7y8Jch…",173.635275,173.855275,"""So1111111111111111111111111111…",173.635275,173.855275,"""67kvwjMKT2nmPvWMQmLraTWK7y8Jch…",2024-12-08 23:45:17,306280192,236.60505,"""Wrapped SOL""",236.60505,"""Wrapped SOL"""
"""63vpc9MAf4Rf598Yb7taVXnnxrMpgv…","""8tqLiEcJZZ5nBRYVR1HAWgChoYJsS7…","""ECXFKACnrfRfNUSx9S5Pz7qjdKuVey…","""So1111111111111111111111111111…",72.6,72.3,"""5Q544fKrFoe6tsEbD7S8EmxGTJYAKt…","""67kvwjMKT2nmPvWMQmLraTWK7y8Jch…",173.855275,174.155275,"""So1111111111111111111111111111…",173.855275,174.155275,"""67kvwjMKT2nmPvWMQmLraTWK7y8Jch…",2024-12-08 23:45:17,306280192,236.60505,"""Wrapped SOL""",236.60505,"""Wrapped SOL"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2JehXx8RRpcbbH9KZ5V74NL9CGpzFQ…","""67yGLFnfA5EnGF3AwzShcfh8VbEujM…","""67yGLFnfA5EnGF3AwzShcfh8VbEujM…","""7vfCXTUXx5WJV5JADk17DUJ4ksgau7…",154.2974,154.3,"""6ZUeThQ9FovzaS8HejAfGW2VqCXwRY…","""BYkeygvLcPRZfiAnahrzgcDu6P5bU4…",22.376461,22.373861,"""7vfCXTUXx5WJV5JADk17DUJ4ksgau7…",22.376461,22.373861,"""BYkeygvLcPRZfiAnahrzgcDu6P5bU4…",2024-12-08 23:56:30,306281831,3995.663841,"""Ether (Portal)""",3995.663841,"""Ether (Portal)"""
"""qKWEYZQH6sX1peoecKEZpUfbFASLFg…","""GFZ31NqdtcEowYsTh4qWWVpFbK11fE…","""Fr8MeQNsjfjz9ybsSfuwydbaZnqggi…","""So1111111111111111111111111111…",2.0,0.0,"""8EyLmmyt8VNqhu8c2NzJ2BCTEZZk4W…","""5eUioESbfewMHtu9DPnMwnoDZyRTsd…",1542.233641,1544.233641,"""So1111111111111111111111111111…",1542.233641,1544.233641,"""5eUioESbfewMHtu9DPnMwnoDZyRTsd…",2024-12-08 23:56:30,306281831,236.60505,"""Wrapped SOL""",236.60505,"""Wrapped SOL"""
"""qKWEYZQH6sX1peoecKEZpUfbFASLFg…","""GFZ31NqdtcEowYsTh4qWWVpFbK11fE…","""Fr8MeQNsjfjz9ybsSfuwydbaZnqggi…","""So1111111111111111111111111111…",2.0,0.0,"""8EyLmmyt8VNqhu8c2NzJ2BCTEZZk4W…","""5eUioESbfewMHtu9DPnMwnoDZyRTsd…",1542.233641,1544.233641,"""So1111111111111111111111111111…",1542.233641,1544.233641,"""5eUioESbfewMHtu9DPnMwnoDZyRTsd…",2024-12-08 23:56:30,306281831,236.60505,"""Wrapped SOL""",236.60505,"""Wrapped SOL"""


In [42]:
df_res.with_columns(
    (
        (pl.col("wallet_prebalance") - pl.col("wallet_postbalance"))
        * pl.col("mint_price")
    ).alias("mint_traded_usd"),
).group_by(
    pl.col("mint_cp_name")
).agg(
    [
        pl.col("mint_traded_usd").sum().alias("mint_traded_usd"),
    ]
).sort("mint_traded_usd")

mint_cp_name,mint_traded_usd
str,f64
"""Orca""",-1.2381e7
"""Star Atlas""",-15547.111614
"""Raydium""",-9426.107819
"""AVAX (Portal)""",-0.193519
"""FTM (Allbridge from Fantom)""",2.674287
"""Bonfida""",52.79417
"""GMT""",429.463065
"""Ether (Portal)""",536123.852796
"""Wrapped SOL""",3.9224e6


In [82]:
account_ops = TokenAccountOpts(
    program_id=Pubkey.from_string(
        "TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA"
    )
    
)
response = client.get_token_accounts_by_owner_json_parsed(
    owner=pubkey,
    opts=account_ops
)

In [83]:
parsed_data = []
for entry in response.value:
    account = entry.account
    parsed_info = account.data.parsed['info']
    parsed_data.append({
        "pubkey": str(entry.pubkey),
        "lamports": account.lamports,
        "program": account.data.program,
        "mint": parsed_info["mint"],
        "owner": parsed_info["owner"],
        "isNative": parsed_info["isNative"],
        "state": parsed_info["state"],
        "tokenAmount": parsed_info["tokenAmount"]["amount"],
        "decimals": parsed_info["tokenAmount"]["decimals"],
        "uiAmount": parsed_info["tokenAmount"]["uiAmount"],
        "uiAmountString": parsed_info["tokenAmount"]["uiAmountString"],
        "space": account.data.space,
        "owner_pubkey": str(account.owner),
        "executable": account.executable,
        "rent_epoch": account.rent_epoch,
    })
df = pl.DataFrame(parsed_data)
df = df.join(
    df_token_list.select(["address", "symbol", "name"]),
    left_on="mint",
    right_on="address",
    how="left"
)
df

pubkey,lamports,program,mint,owner,isNative,state,tokenAmount,decimals,uiAmount,uiAmountString,space,owner_pubkey,executable,rent_epoch,symbol,name
str,i64,str,str,str,bool,str,str,i64,f64,str,i64,str,bool,u64,str,str
"""3WYgKGMaUndUi7H36sMmY6jDcxThN1…",2039280,"""spl-token""","""35FYCTQFA1m18nJ3GgJ8jzd4HgnmZf…","""3LkGTjNsF2zWc2ddBPHYyEJJZKqbdH…",false,"""initialized""","""69000000""",6,69.0,"""69""",165,"""TokenkegQfeZyiNwAJbNbGKPFXCWuB…",false,18446744073709551615,null,null
"""74Lb7TrKYmmYuDu23DRXrHCVrajGHF…",2039280,"""spl-token""","""4gHpM7xtQwoo2mpTyHJAT5bgMkT5mW…","""3LkGTjNsF2zWc2ddBPHYyEJJZKqbdH…",false,"""initialized""","""5000000""",4,500.0,"""500""",165,"""TokenkegQfeZyiNwAJbNbGKPFXCWuB…",false,18446744073709551615,null,null
"""9AU8XmESb8apXFM27d36ngzUiwwcHi…",2039280,"""spl-token""","""EVAknb8Rc3GsZC4yz36C9qXDUJbJDt…","""3LkGTjNsF2zWc2ddBPHYyEJJZKqbdH…",false,"""initialized""","""78333000000""",6,78333.0,"""78333""",165,"""TokenkegQfeZyiNwAJbNbGKPFXCWuB…",false,18446744073709551615,null,null
"""4MqbWh7bu4MaczvMNhYTaBVaZvhyaU…",2039280,"""spl-token""","""7ZqzGzTNg5tjK1CHTBdGFHyKjBtXdf…","""3LkGTjNsF2zWc2ddBPHYyEJJZKqbdH…",false,"""initialized""","""47801312755""",6,47801.312755,"""47801.312755""",165,"""TokenkegQfeZyiNwAJbNbGKPFXCWuB…",false,18446744073709551615,null,null
"""HHCFFr7GJ7eFFGPCa5ZpavfYtWLUZs…",2039280,"""spl-token""","""28ejJfWvsvEYQadjDCHu9XWTNUo224…","""3LkGTjNsF2zWc2ddBPHYyEJJZKqbdH…",false,"""initialized""","""0""",9,0.0,"""0""",165,"""TokenkegQfeZyiNwAJbNbGKPFXCWuB…",false,18446744073709551615,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""95oML6mnvdohxRVpBZp87KbcqCccRZ…",2039280,"""spl-token""","""3M2vepByfZTG6xUSmFidvSCFzHWFoV…","""3LkGTjNsF2zWc2ddBPHYyEJJZKqbdH…",false,"""initialized""","""10000000""",6,10.0,"""10""",165,"""TokenkegQfeZyiNwAJbNbGKPFXCWuB…",false,18446744073709551615,null,null
"""76JuoBzkay4DyH5xGPzMriDwGPiJbw…",2039280,"""spl-token""","""HxBbHRcyQf69cFLox5ckGtxAKfzdgi…","""3LkGTjNsF2zWc2ddBPHYyEJJZKqbdH…",false,"""initialized""","""353939499300""",9,353.939499,"""353.9394993""",165,"""TokenkegQfeZyiNwAJbNbGKPFXCWuB…",false,18446744073709551615,null,null
"""EbqW46Aa15Dde6xQ44txp3JhFV84Zq…",2039280,"""spl-token""","""BCP3tdCpDGyNzcgi8s2abF4EqVzjGQ…","""3LkGTjNsF2zWc2ddBPHYyEJJZKqbdH…",false,"""initialized""","""10000000""",6,10.0,"""10""",165,"""TokenkegQfeZyiNwAJbNbGKPFXCWuB…",false,18446744073709551615,null,null


In [84]:
transaction_signatures = []
before = None
while True:
    response = client.get_signatures_for_address(pubkey, before=before)
    if len(response.value) == 0:
        break
    transaction_signatures.extend(response.value)
    before = response.value[-1].signature

SolanaRpcException: 

In [85]:
signatures = [
    t.signature for t in transaction_signatures
    if t.err is None
]
signatures[0:2]

[Signature(
     3CeFzmKDausZgWvJX5yxGjRGv54MvYF1EGHbwZZkNnvN6WYL7t871TmJ6e7zdrXkTswQZKYVPchPouC69DgbdhL1,
 ),
 Signature(
     43eZD6ZqNH2BXNfzHvZJ4EwFFoZDmur5K59bkfk5SVu1DmNbr3xEkf4WPSHPwq67jebF6jZ2EvUmBCUrSnxvL8tR,
 )]

In [ ]:
from atlas_sol import AtlasSol
from tqdm import tqdm
CHAINSTAKE_SOLONA_HTTPS = "https://solana-mainnet.core.chainstack.com/2fa69914c087050cc7b9887511cd7d35"
CHAINSTAKE_SOLONA_WS = "wss://solana-mainnet.core.chainstack.com/2fa69914c087050cc7b9887511cd7d35"
asol = AtlasSol(
    url="https://api.mainnet-beta.solana.com"
)
tx = asol.getTransaction(signatures)

In [90]:
tx.__dict__

{'owner': '3musSxbKab3GgXNhKpmzspZhvq1BAVgmEJ7Yz5HBdxmB',
 'account': 'kfM4qbiwE4xN1i7QcS8DFtyh9V3wRdrQrRCHXoCbpvZ',
 'mint': 'A8C3xuqscfmyLrte3VmTqrAq8kgMASius9AFNANwpump',
 'wallet_prebalance': 371805.731861,
 'wallet_postbalance': 372695.840284,
 'owner_cp': '3LkGTjNsF2zWc2ddBPHYyEJJZKqbdHJDgfjztxnjwL5R',
 'account_cp': '37BkVCjp7N4ni4uBde4731VLzTKKQVLPHsefP5ZXs94t',
 'wallet_prebalance_cp': 12652.589319,
 'wallet_postbalance_cp': 11762.480896,
 'mint_cp': 'So11111111111111111111111111111111111111112',
 'wallet_prebalance_sold': 489.85567842,
 'wallet_postbalance_sold': 488.160230902,
 'account_sold': 'BUmsGx7R5dhbmG6P7aVeZUqwob99xcetAbZDsFz5CkAo'}

In [159]:
from pythclient.pythclient import PythClient
from pythclient.pythaccounts import PythPriceAccount, PythPriceStatus
from pythclient.solana import SolanaClient, SolanaPublicKey, PYTHNET_HTTP_ENDPOINT, PYTHNET_WS_ENDPOINT
from solana.rpc.api import Client  # Using the synchronous client
import polars as pl  # Assuming polars for DataFrame handling
from pythclient.utils import get_key

solana_network="devnet"
use_program = False
v2_first_mapping_account_key = get_key("pythnet", "mapping")
v2_program_key = get_key("pythnet", "program")
pyth_client = PythClient(
    first_mapping_account_key=v2_first_mapping_account_key,
    program_key=v2_program_key if use_program else None,
    solana_endpoint=PYTHNET_HTTP_ENDPOINT, # replace with the relevant cluster endpoints
    solana_ws_endpoint=PYTHNET_WS_ENDPOINT # replace with the relevant cluster endpoints
)
await pyth_client.refresh_all_prices()
products = await pyth_client.get_products()
len(products)

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x0000023C7ECD6C90>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x0000023C7FB18BC0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x0000023C7F4588C0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x0000023C7F4587A0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x0000023C6E003B60>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x0000023C6E003E60>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x0000023C7F364DA0>


592

In [163]:
prices = await products[3].get_prices()
for _, pr in prices.items():
    print(pr)

PythPriceAccount Crypto.ALGO/USD PythPriceType.PRICE (HqFyq1wh1xKvL7KDqqT7NJeSPdAqsDqnmBisUC2XdXAX)


In [164]:
pr.__dict__

{'key': HqFyq1wh1xKvL7KDqqT7NJeSPdAqsDqnmBisUC2XdXAX,
 'solana': <pythclient.solana.SolanaClient at 0x23c6e238590>,
 'slot': 182831082,
 'lamports': 110524800000,
 'product': PythProductAccount Crypto.ALGO/USD (4JCKSFQXT1Nv89fMX5reV7LXJUUxzb6rUAqMiC8oS9vf),
 'price_type': <PythPriceType.PRICE: 1>,
 'exponent': -8,
 'num_components': 31,
 'last_slot': 182831082,
 'valid_slot': 182831081,
 'product_account_key': 4JCKSFQXT1Nv89fMX5reV7LXJUUxzb6rUAqMiC8oS9vf,
 'next_price_account_key': None,
 'aggregate_price_info': PythPriceInfo status PythPriceStatus.TRADING price 0.50435733,
 'price_components': [PythPriceComponent(publisher_key=7YQg8Tz9KHKsg7yHiAFRBsDkLoKvZbMXt7VbW44F7QM, last_aggregate_price_info=PythPriceInfo status PythPriceStatus.TRADING price 0.5045135000000001, latest_price_info=PythPriceInfo status PythPriceStatus.TRADING price 0.5045135000000001, exponent=-8),
  PythPriceComponent(publisher_key=JTmFx5zX9mM94itfk2nQcJnQQDPjcv4UPD7SYj6xDCV, last_aggregate_price_info=PythPriceInfo

In [166]:
price = pr.aggregate_price_info.price
price

0.50435733

In [ ]:
account_key = SolanaPublicKey("FsSM3s38PX9K7Dn6eGzuE29S2Dsk1Sss1baytTQdCaQj")
solana_client = SolanaClient(endpoint=PYTHNET_HTTP_ENDPOINT, ws_endpoint=PYTHNET_WS_ENDPOINT)
price: PythPriceAccount = PythPriceAccount(account_key, solana_client)

await price.update()
print("DOGE/USD is", price.aggregate_price, "±", price.aggregate_price_confidence_interval)

32.13808

In [76]:
from pythclient.pythclient import PythClient
from solana.rpc.api import Client  # Using the synchronous client
import polars as pl  # Assuming polars for DataFrame handling
from pythclient.utils import get_key

solana_network="devnet"
async with PythClient(
    first_mapping_account_key=get_key(solana_network, "mapping"),
    program_key=None
) as c:
    await c.refresh_all_prices()
    products = await c.get_products()
    dfs = []
    for p in products:
        prices = await p.get_prices()
        for _, pr in prices.items():
            dfs.append(
                {
                    "price_type": pr.price_type,
                    "aggregate_price_status": pr.aggregate_price_status,
                    "aggregate_price": pr.aggregate_price,
                    "aggregate_price_confidence_interval": pr.aggregate_price_confidence_interval,
                    **p.attrs
                }
            )
    df_prices = pl.DataFrame(dfs)
df_prices

price_type,aggregate_price_status,aggregate_price,aggregate_price_confidence_interval,symbol,asset_type,base,description,generic_symbol,quote_currency,weekly_schedule,cms_symbol,country,cqs_symbol,nasdaq_symbol,tenor
object,object,null,null,str,str,str,str,str,str,str,str,str,str,str,str
PythPriceType.PRICE,PythPriceStatus.UNKNOWN,null,null,"""Crypto.BCH/USD""","""Crypto""","""BCH""","""BITCOIN CASH / US DOLLAR""","""BCHUSD""","""USD""","""America/New_York,O,O,O,O,O,O,O""",null,null,null,null,null
PythPriceType.PRICE,PythPriceStatus.UNKNOWN,null,null,"""Crypto.LTC/USD""","""Crypto""","""LTC""","""LITECOIN / US DOLLAR""","""LTCUSD""","""USD""","""America/New_York,O,O,O,O,O,O,O""",null,null,null,null,null
PythPriceType.PRICE,PythPriceStatus.UNKNOWN,null,null,"""Equity.US.AAPL/USD""","""Equity""","""AAPL""","""APPLE INC / US DOLLAR""",null,"""USD""","""America/New_York,9:30-16:00,9:…","""AAPL""","""US""","""AAPL""","""AAPL""",null
PythPriceType.PRICE,PythPriceStatus.UNKNOWN,null,null,"""Equity.US.TSLA/USD""","""Equity""","""TSLA""","""TESLA INC / US DOLLAR""",null,"""USD""","""America/New_York,9:30-16:00,9:…","""TSLA""","""US""","""TSLA""","""TSLA""",null
PythPriceType.PRICE,PythPriceStatus.UNKNOWN,null,null,"""Equity.US.QQQ/USD""","""Equity""","""QQQ""","""INVESCO QQQ TRUST SERIES 1 / U…",null,"""USD""","""America/New_York,9:30-16:00,9:…","""QQQ""","""US""","""QQQ""","""QQQ""",null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
PythPriceType.PRICE,PythPriceStatus.UNKNOWN,null,null,"""Crypto.ER.WSTETH/STETH""","""Crypto Exchange Rate""","""WSTETH""","""WSTETH EXCHANGE RATE""","""WSTETHSTETH""","""STETH""",null,null,null,null,null,null
PythPriceType.PRICE,PythPriceStatus.UNKNOWN,null,null,"""Crypto.ER.WUSDM/USDM""","""Crypto Exchange Rate""","""WUSDM""","""WUSDM EXCHANGE RATE""","""WUSDMUSDM""","""USDM""",null,null,null,null,null,null
PythPriceType.PRICE,PythPriceStatus.UNKNOWN,null,null,"""Crypto.Index.GML2/USD""","""Crypto Index""","""GML2""","""GMCI L2 / US DOLLAR""","""GML2USD""","""USD""","""America/New_York,O,O,O,O,O,O,O""",null,null,null,null,null


In [168]:
CHAINSTAKE_SOLONA_HTTPS = "https://solana-mainnet.core.chainstack.com/2fa69914c087050cc7b9887511cd7d35"
CHAINSTAKE_SOLONA_WS = "wss://solana-mainnet.core.chainstack.com/2fa69914c087050cc7b9887511cd7d35"

In [169]:
import requests

payload = {
    "id": 1,
    "jsonrpc": "2.0",
    "method": "getAccountInfo",
    "params": [
        wallet_address,
        {
            "encoding": "jsonParsed",
            "commitment": "finalized"
        }
    ]
}
headers = {
    "accept": "application/json",
    "content-type": "application/json"
}

response = requests.post(CHAINSTAKE_SOLONA_HTTPS, json=payload, headers=headers)
print(response.text)

{"jsonrpc":"2.0","result":{"context":{"apiVersion":"2.0.18","slot":305880263},"value":{"data":["","base64"],"executable":false,"lamports":674862200007,"owner":"11111111111111111111111111111111","rentEpoch":18446744073709551615,"space":0}},"id":1}

